# Spectral Quantum Tomography

*Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline
A central challenge on the path towards large-scale quantum computing is the engineering of high-quality quantum gates. A method that accurately and reliably characterize quantum gates is desirable.
Spectral Quantum Tomography (SQT) is a powerful technique for extracting spectral information from noisy gates, and is resistant to state-preparation and measurement (SPAM) errors. The main advantage of spectral quantum tomography is its simplicity, requiring only the (repeated) application of a single noisy gate $\mathcal{N}$, as opposed to the application of a large set of gates as in randomized benchmarking, gate-set tomography, and robust tomography. Another feature of spectral tomography is that it can be used to extract signatures of non-Markovianity.

This tutorial introduces Spectral Quantum Tomography (SQT), covering its theory and implementation on [Baidu Quantum Platform](https://quantum.baidu.com/).


## Theory

### Pauli transfer matrix representation
The spectral information of a noisy gate $\mathcal{N}$, which approximates some target unitary $U$, is given by the eigenvalues of the so-called Pauli Transfer Matrix (PTM) representing $[\mathcal{N}]$. A unitary gate $U$ can be considered as a trace-preserving completely positive (TPCP) maps. For $n$-qubits system, we construct the normalized set of Pauli matrices $P_i$ ($i = 0, \dots, N$ with $N+1 = 4^n$), e.g., $P_0 = \frac{1}{\sqrt{2}}I$. For a TPCP map $\mathcal{N}$ acting on $n$ qubits, the PTM is then define as

$$
    [\mathcal{N}]_{ij} = \textrm{Tr}\left[ P_i \mathcal{N}(P_{j})\right],\; i, j = 0, \dots, N.
$$

The form of the Pauli transfer matrix $[\mathcal{N}]$ is

$$
    [\mathcal{N}] = \begin{pmatrix}
        1 & 0 \\ \mathbf{n} & T^\mathcal{N}
    \end{pmatrix},
$$

where $T^\mathcal{N}$ is a real $N \times N$ matrix (due to expanding in terms of Pauli basis) and $\mathbf{n}$ is a $N$-dimensional column vector.
The $1$ and $0$s in the top row of the Pauli tranfer matrix are due to the fact that $\mathcal{N}$ is trace-preserving.
For an unital $\mathcal{N}$ that obeys $\mathcal{N}(I) = I$, the vector $\mathbf{n} = \mathbf{0}$.


### Spectral tomography method
We model state-preparation errors as a prefect preparation step followed by an unknown TPCP map $\mathcal{N}_{prep}$. Similarly, measurement errors are modeled by a perfect measurement preceded by an unknown TPCP map $\mathcal{N}_{meas}$. We assume that, when we apply the targeted gate $k$ times, an accurate model of the resulting noisy dynamics is $\mathcal{N}^k$.
For $k = 0,1,\dots,K$ for some fixed $K$:

$$
    g(k) = \sum_i \textrm{Tr} \left[ P_i \mathcal{N}_{meas} \circ \mathcal{N}^k \circ \mathcal{N}_{prep}(P_i)\right].
$$

The experiments steps are as follows:
1. Picking a traceless $n$-qubit Pauli $P_i$. (Note we don't chose $I$ as the input state)
2. Preparing an $n$-qubit input state in one of the $2^n$ basis state corresponding to this chosen Pauli. (Note: this process corresponding to $\mathcal{N}_{prep}$, and we need to do spectral decomposition for each Pauli basis first)
3. Applying the gate $k$ times and measuring in the same chosen Pauli basis. (Note: this process corresponding to and $\mathcal{N}^k$ and $\mathcal{N}_{meas}$)
4. Repeating (1-3) over different Paulis basis states, and experiments to get good statistics.

Then we can see how $g(k)$ depends on the eigenvalues of the matrix $T$. When there are no state-preparation and measurement (SPAM) errors, that is, $\mathcal{N}_{meas}$ and $\mathcal{N}_{prep}$ are identity channels, we have

$$
    g^{ideal}(k) = \sum_{i} \textrm{Tr} \left[ P_i \mathcal{N}^k(P_i)\right] = \sum_i (T^k)_{ii}
    = \textrm{Tr} \left[T^k\right] = \sum_{i=1}^{N} \lambda_i,
$$

where $\left\{ \lambda_i \right\}$ are the eigenvalues of $T$ (Hint: the last step in this equation needs $T$ is diagonalizable).  
When $\mathcal{N}_{meas}$ and $\mathcal{N}_{prep}$ are not identity channels, we have

$$
\begin{aligned}
    g(k) &= \textrm{Tr} \left[T_{meas} T^k T_{prep}\right] = \textrm{Tr} \left[T_{meas} VD^kV^{-1} T_{prep}\right] \\
    &= \textrm{Tr} \left[V^{-1} T_{prep} T_{meas} V D^k\right] \\
    &= \textrm{Tr} \left[A_{SPAM} D^k\right] \\
    &= \sum_i A_{i} \lambda_i^k.
\end{aligned}
$$

where $T_{meas}$ and $T_{prep}$ are, respectively, the $T$-submatrix of the PTM of $\mathcal{N}_{meas}$ and $\mathcal{N}_{prep}$. Here we assume that $T = VDV^{-1}$ is diagonalizable and the matrix $A_{SPAM} = V^{-1} T_{prep} T_{meas} V$, $A_{i} = [A_{SPAM}]_{ii}$. 

### Matrix-pencil method
Matrix-pencil method is kind of classical signal-processing method that reconstructs, from the (noisy) signal $g(k)=\sum_i A_{i}\lambda_i^k$ for $k=0,\dots,K$, an estimate for the eigenvalues $\lambda_i$ and the amplitudes $A_{i}$. We require at least $K \geq 2N-2$ in order to determine the eigenvalues accurately, where $N = 4^n - 1$ for $n$-qubits system.
Then we introduce an important parameter called pencil parameter $L$, which determine the shape of matrix $Y$, when $Y$ is a $(K-L+1) \times (L+1)$-dimensional matrix,

$$
\begin{align}
    Y = \begin{pmatrix}
        g(0) & g(1) & \cdots & g(L) \\
        g(1) & g(2) & \cdots & g(L+1) \\
        g(2) &  & & \vdots \\
        \vdots & & \ddots & \vdots \\
        g(K-L) & \cdots & \cdots & g(K)
    \end{pmatrix} = 
    \sum_{i=1}^{N} A_i \begin{pmatrix}
        1 & \lambda_i & \cdots & \lambda_i^L \\
        \lambda_i & \lambda_i^2 & \cdots & \lambda_i^{L+1} \\
        \lambda_i^2 & & & \vdots \\
        \vdots & & \ddots & \vdots \\
        \lambda_i^{K-L} & \cdots & \cdots & \lambda_i^{K}
    \end{pmatrix}.
\end{align}
$$

Consider two submatrices of $Y$,

$$
\begin{align}
    G_0 = \begin{pmatrix}
        g(0) & g(1) & \cdots & g(L-1) \\
        g(1) & g(2) & \cdots & g(L) \\
        g(2) &  & & \vdots \\
        \vdots & & \ddots & \vdots \\
        g(K-L) & \cdots & \cdots & g(K-1)
    \end{pmatrix},\;
    G_1 = \begin{pmatrix}
        g(1) & g(2) & \cdots & g(L) \\
        g(2) & g(3) & \cdots & g(L+1) \\
        g(3) & & & \vdots \\
        \vdots & & \ddots & \vdots \\
        g(K-L+1) & \cdots & \cdots & g(K)
    \end{pmatrix}.
\end{align}
$$

- for noiseless data  
One can write

$$
\begin{align}
    G_1 &= \Lambda_1 R \Lambda_0 \Lambda_2, \\
    G_0 &= \Lambda_1 R \Lambda_2.
\end{align}
$$

Now consider the matrix pencil

$$
\begin{align}
    G_1 - \lambda G_0 = \Lambda_1 R (\Lambda_0 - \lambda I) \Lambda_2.
\end{align}
$$

Equivalently, the problem of solving for $\lambda_i$ can be cast as an ordinary eigenvalue problem,

$$
\begin{align}
    G_0^+ G_1 - \lambda I = 0, 
\end{align}
$$

where $G_0^+$ is the Moore-Penrose pseudo-inverse of $G_0$. This, in turn, is defined as

$$
\begin{align}
    G_0^+ = (G_0^\dagger G_0)^{-1} G_0^\dagger.
\end{align}
$$


- for noise data  
A singular-value decomposition (SVD) of the matrix Y is carried out as

$$
\begin{align}
    Y = U \Sigma V^\dagger.
\end{align}
$$

Here, $U$ and $V$ are unitary matrices, composed of the eigenvectors of $YY^\dagger$ and $Y^\dagger Y$, 
respectively, and $\Sigma$ is a diagonal matrix containing the singular values of $Y$.
We next consider the "filtered" matrix, $V^{'}$, 
constructed so that it contains only $N$ dominant right-singular vectors of $V$:

$$
\begin{align}
    V = [v_1, v_2, \dots, v_N].
\end{align}
$$

The right-singular vectors from $N+1$ to L, 
corresponding to the small singular values, are discarded. 
Therefore,

$$
\begin{align}
    G_0 &= U \Sigma^{'} V_0^{'\dagger}, \\
    G_1 &= U \Sigma^{'} V_1^{'\dagger},
\end{align}
$$

where $V_0^{'\dagger}$ is obtained from $V^{'}$ with the last row of $V^{'}$ deleted, $V_1^{'\dagger}$ is obtained from $V^{'}$ with the first row of $V^{'}$ deleted, and $\Sigma^{'}$ is obtained from the $N$ columns of $\Sigma$ corresponding to the $N$ dominant singular values. The eigenvalues of the following matrix,

$$
\begin{align}
    G_1 - \lambda G_0 \Rightarrow G_0 G_1^+ - \lambda I. 
\end{align}
$$

are equivalent to the eigenvalues of the following matrix,

$$
\begin{align}
    V_1^{'\dagger} - \lambda V_0^{'\dagger} \Rightarrow V_0^{'\dagger} \left(V_1^{'\dagger}\right)^+ - \lambda I.
\end{align}
$$



## Practice

### Single-qubit gate case
First, we import the necessary libraries. 

In [1]:
import numpy as np
import QCompute
import qcompute_qep.tomography as tomography
import qcompute_qep.quantum.pauli as pauli
import qcompute_qep.utils.circuit

We construct the ideal $R_x(\frac{1}{4} \pi)$ gate first, and calculate the corresponding unitary matrix and PTM representation form, then obtain the ideal eigenvalues of $T^\mathcal{N}$. 

In [ ]:
qp = QCompute.QEnv()  # qp is short for "quantum program", instance of QProgram
qp.Q.createList(1)

QCompute.RX(np.math.pi / 4)(qp.Q[0])

# Compute numerically the ideal R_x for reference
ideal_rx = qcompute_qep.utils.circuit.circuit_to_unitary(qp)
ideal_ptm = pauli.unitary_to_ptm(ideal_rx).data
print("the ideal Rotation x is \n", ideal_rx)
print("the ideal PTM of rotation x is \n", ideal_ptm)

# calculate the eigenvalues of PTM representation
ideal_eigenvalues, _ = np.linalg.eig(ideal_ptm[1:, 1:])
print("the ideal eigenvalues of PTM representation is \n", ideal_eigenvalues)


Then we set the quantum computer (instance of QComputer). The QuantumComputer can be a simulator or a hardware interface. The rest is simple, we initialize a SpectralTomography instance,  call the tomography procedure and obtain the eigenvalues.

In [ ]:
# Please login the "Quantum Leaf" platform (https://quantum-hub.baidu.com/) to get Token
QCompute.Define.hubToken = "Token"

qc = QCompute.BackendName.CloudBaiduQPUQian

st = tomography.SpectralTomography()
noisy_eigvals = st.fit(qp, qc, k=50, l=30)

print("the eigenvalues we estimate is ", noisy_eigvals)

We can visualize the data with matplotlib. Because we use local simulator, our estimate is very close to the ideal eigenvalues.

In [ ]:
import matplotlib.pyplot as plt
from cmath import *

ax = plt.subplot(polar=True)

ax.set_rlim(0.99, 1.01)
noisy_data = np.zeros((2, np.size(noisy_eigvals)), dtype=float)
ideal_data = np.zeros((2, np.size(ideal_eigenvalues)), dtype=float)
for i, val in enumerate(noisy_eigvals):
    noisy_data[:, i] = np.asarray(polar(val))

for i, val in enumerate(ideal_eigenvalues):
    ideal_data[:, i] = np.asarray(polar(val))

ax.scatter(noisy_data[1, :], noisy_data[0, :], c='blue', label='noisy')
ax.scatter(ideal_data[1, :], ideal_data[0, :], c='red', label='ideal')
plt.legend()
plt.show()


![SQT](./figures/sqt-rx-output.png "Figure 1: We visualize the result of single-qubit case.")


### Two-qubit gate case

Similarly, we set up the quantum program for the CNOT gate and calculate the ideal eigenvalues of $T^\mathcal{N}$.

In [ ]:
qp = QCompute.QEnv()  # qp is short for "quantum program", instance of QProgram
qp.Q.createList(2)

# Manually decompose the CNOT gate using the CZ gate, where CNOT: q1 -> q0
QCompute.H(qp.Q[0])
QCompute.CZ(qp.Q[1], qp.Q[0])
QCompute.H(qp.Q[0])

# Compute numerically the ideal CNOT for reference
ideal_cnot = qcompute_qep.utils.circuit.circuit_to_unitary(qp)
ideal_ptm = pauli.unitary_to_ptm(ideal_cnot).data
print("the ideal CNOT is \n", ideal_cnot)
print("the ideal PTM of CNOT is \n", ideal_ptm[1:, 1:])

# calculate the eigenvalues of PTM representation
ideal_eigenvalues, _ = np.linalg.eig(ideal_ptm)
print("the ideal eigenvalues of PTM representation is \n", ideal_eigenvalues)

qc = QCompute.BackendName.CloudBaiduQPUQian

We know that the $T^\mathcal{N}$ of CNOT gate in principle contain $15$ eigenvalues (which are degenerate). Since we can vary the number of eigenvalues we use to fit the signal $g(k)$ to see whether a different choice than $N$ gives a significantly better fit. Here we set $N=2$ in code.

In [ ]:
# Initialize a ProcessTomography instance
st = tomography.SpectralTomography()
# Call the tomography procedure and obtain the noisy CNOT gate
noisy_eigvals = st.fit(qp, qc, k=50, l=30, N=2)

We can visualize the data with matplotlib. Because we use local simulator, our estimate is very close to the ideal eigenvalues.

In [ ]:
import matplotlib.pyplot as plt
from cmath import *

ax = plt.subplot(polar=True)

ax.set_rlim(0.99, 1.01)
noisy_data = np.zeros((2, np.size(noisy_eigvals)), dtype=float)
ideal_data = np.zeros((2, np.size(ideal_eigenvalues)), dtype=float)

for i, val in enumerate(noisy_eigvals):
    noisy_data[:, i] = np.asarray(polar(val))

for i, val in enumerate(ideal_eigenvalues):
    ideal_data[:, i] = np.asarray(polar(val))

# print("ideal data:\n", ideal_data)
# print("noisy data:\n", noisy_data)

ax.scatter(noisy_data[1, :], noisy_data[0, :], c='blue', label='noisy')
ax.scatter(ideal_data[1, :], ideal_data[0, :], c='red', label='ideal')
plt.legend()
plt.show()



![](./figures/sqt-cnot-output.png "Figure 2: We visualize the result of two-qubit case. ")


## Summary
This tutorial describes how to use Spectral Tomography method to extract the eigenvalues of a quantum channel.

## References
[1] Helsen, Jonas, Francesco Battistel, and Barbara M. Terhal. "Spectral quantum tomography." [npj Quantum Information](https://www.nature.com/articles/s41534-019-0189-0) 5.1 (2019): 1-11.

[2] Sarkar, Tapan K., and Odilon Pereira. "Using the matrix pencil method to estimate the parameters of a sum of complex exponentials." [IEEE Antennas and Propagation Magazine](https://ieeexplore.ieee.org/abstract/document/370583/) 37.1 (1995): 48-55.